<a href="https://colab.research.google.com/github/elkhayyat17/chat-with-pdfs-mistral-7b/blob/main/chat_with_your_pdfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install -q transformers


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=7138149 sha256=35cc0dde2eee5a33b8c8165068fb8bdd58e4205590ebf19346649997fd2d2e2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ol4p20e0/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [4]:
!pip install -q llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.4/923.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


Enable text wrapping so we don't have to scroll horizontally and create a function to flush CUDA cache.

In [17]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [7]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


download document


In [6]:
#!wget url of pdf
#or upload  pdf to content/Data
!wget https://solar-center.stanford.edu/teachers/Our-Solar-System-Book.pdf

--2023-12-04 16:20:51--  https://solar-center.stanford.edu/teachers/Our-Solar-System-Book.pdf
Resolving solar-center.stanford.edu (solar-center.stanford.edu)... 171.64.103.240
Connecting to solar-center.stanford.edu (solar-center.stanford.edu)|171.64.103.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5123417 (4.9M) [application/pdf]
Saving to: ‘Our-Solar-System-Book.pdf’

Our-Solar-System-Bo 100%[===================>]   4.89M  11.6MB/s    in 0.4s    

2023-12-04 16:20:51 (11.6 MB/s) - ‘Our-Solar-System-Book.pdf’ saved [5123417/5123417]



In [8]:
!mkdir Data
!mv "Our-Solar-System-Book.pdf" "/content/Data"



In [9]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [5]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:18, 224.97it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [10]:
!pip install  -q "sentence-transformers" "langchain"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.6 MB/s eta 0:00:00


In [11]:
!pip install -q "llama_index"

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
  #HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

)


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [13]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [15]:
query_engine = index.as_query_engine()


In [18]:
response =  query_engine.query("summarize this book")
print(response)

Llama.generate: prefix-match hit


 Based on the given context information, it appears that the book is about our solar system and beyond. The book covers topics such as the moons of Uranus and Neptune, including Triton, as well as the Kuiper Belt, dwarf planets, trans-Neptunian objects, comets, interplanetary dust particles, and the Sun's heliosphere. Additionally, the book discusses where we are in the universe and our immense universe. However, without further information about the book's content or structure, it is difficult to provide a more detailed summary.


In [20]:
response =  query_engine.query("what do you know about moons of Uranus")
print(response)

Llama.generate: prefix-match hit


 According to the provided context information, Uranus has 27 known moons. However, it is important to note that this information may have been obtained from NASA Voyager 2's flyby of the planet in 1986 and therefore may not be up-to-date. Additionally, the context information does not provide specific details about each moon such as their names or characteristics.


In [21]:
import re
document_info = str(response.metadata)
resorce=str(re.findall(r"'page_label': '[^']*', 'file_name': '[^']*'", document_info))
print(resorce)

["'page_label': '35', 'file_name': 'Our-Solar-System-Book.pdf'", "'page_label': '36', 'file_name': 'Our-Solar-System-Book.pdf'"]


In [29]:
import panel as pn
import param
import re
pn.extension()
class cbfs(param.Parameterized):
    answer = param.String("")
    resource =  param.List([])
    db_query  = param.String("")



    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.panels_ = []



    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('**User:**', pn.pane.Markdown("", width=600)), scroll=True, styles={'background-color': '#FAF5E4'})
        response =  query_engine.query(query)
        ans=response.response
        document_info = str(response.metadata)
        self.db_query =str(re.findall(r"'page_label': '[^']*', 'file_name': '[^']*'", document_info))




        self.answer =ans

        self.panels.extend([
            pn.Row('**User:**', pn.pane.Markdown(query, width=600,styles={'background-color': '#FAF5E4'})),
            pn.Row('**ChatBot:**', pn.pane.Markdown(self.answer  , width=600, styles={'background-color': '#ffffff'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)


    @param.depends('db_query ', )
    def resource(self):
      if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#ffffff'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )


      self.panels_.extend([
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#ffffff'})),

            pn.pane.Str(self.db_query )
        ])
      return pn.WidgetBox(*self.panels_,scroll=True)





In [30]:
cb = cbfs()


inp = pn.widgets.TextInput( placeholder='Massage your bot …', styles={'background-color': '#ffffff'},width=500)

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(

    pn.panel(conversation,  loading_indicator=True, height_policy="fit", styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),
    pn.Row(inp, styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),
)

tab2 = pn.Column(

    pn.panel(cb.resource, styles={'background-color': '#ffffff'}),
    pn.layout.Divider(),

)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Chat With Your PDFs_Bot'), styles={'background-color': '#ffffff'},width=500),
    pn.Tabs(('Conversation', tab1), ('Resources', tab2), styles={'background-color': '#ffffff'})
)
dashboard

Column
    [0] Row(styles={'background-color': '...}, width=500)
        [0] Markdown(str)
    [1] Tabs(styles={'background-color': '...})
        [0] Column
            [0] ParamFunction(function, _pane=WidgetBox, defer_load=False, height_policy='fit', loading_indicator=True, styles={'background-color': '...})
            [1] Divider()
            [2] Row(styles={'background-color': '...})
                [0] TextInput(placeholder='Massage your bot …', styles={'background-color': '...}, width=500)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False, styles={'background-color': '...})
            [1] Divider()

Llama.generate: prefix-match hit
